In [1]:
import uproot
import awkward as ak

# Define your file location
#FILE_LOC = "/data/atlas/data/rho_delta/rho_small.root"
FILE_LOC = "/fast_scratch_1/jbohm/cell_particle_deposit_learning/delta/delta_root_files/delta_full.root"
#FILE_LOC = "/data/atlas/data/rho_delta/delta.root"


events = uproot.open(FILE_LOC+":EventTree")
cell_geo_tree = uproot.open("/data/atlas/data/rho_delta/rho_small.root"+":CellGeo")
queryTree = cell_geo_tree


for key in queryTree.keys():
    print(key)

node_feature_names = cell_geo_tree.keys()[1:7] # 'cell_geo_sampling', 'cell_geo_eta', 'cell_geo_phi', 'cell_geo_rPerp', 'cell_geo_deta', 'cell_geo_dphi'




runNumber
eventNumber
lumiBlock
coreFlags
mcEventNumber
mcChannelNumber
mcEventWeight
nTruthPart
G4PreCalo_n_EM
G4PreCalo_E_EM
G4PreCalo_n_Had
G4PreCalo_E_Had
truthVertexX
truthVertexY
truthVertexZ
truthPartPdgId
truthPartStatus
truthPartBarcode
truthPartPt
truthPartE
truthPartMass
truthPartEta
truthPartPhi
nTrack
trackPt
trackP
trackMass
trackEta
trackPhi
trackTruthParticleIndex
trackVisibleCalHitCaloEnergy
trackFullCalHitCaloEnergy
trackSubtractedCaloEnergy
trackNumberOfPixelHits
trackNumberOfSCTHits
trackNumberOfPixelDeadSensors
trackNumberOfSCTDeadSensors
trackNumberOfPixelSharedHits
trackNumberOfSCTSharedHits
trackNumberOfPixelHoles
trackNumberOfSCTHoles
trackNumberOfInnermostPixelLayerHits
trackNumberOfNextToInnermostPixelLayerHits
trackExpectInnermostPixelLayerHit
trackExpectNextToInnermostPixelLayerHit
trackNumberOfTRTHits
trackNumberOfTRTOutliers
trackChiSquared
trackNumberDOF
trackD0
trackZ0
trackEta_PreSamplerB
trackPhi_PreSamplerB
trackEta_PreSamplerE
trackPhi_PreSamplerE
t

In [2]:
n_truth_part = events["nTruthPart"].array()


num_particles = 8
first_index = ak.firsts(ak.where(n_truth_part >= num_particles))

if ak.is_valid(first_index):
    print(f"Event {first_index} has {n_truth_part[first_index]} truth particles.")
else:
    print(f"No event with {num_particles} or more truth particles was found.")

Event [161730] has [8] truth particles.


In [3]:
import numpy as np
def spherical_to_cartesian(rs, phis, thetas):
    # convert to x, y, z
    x, y, z = [], [], []

    for i, r in enumerate(rs):
        x.append(r*np.cos(phis[i]))
        y.append(r*np.sin(phis[i]))
        z.append( r/np.tan(thetas[i]))

    return np.array(x), np.array(y), np.array(z)

In [5]:
# Define your queries
queries = ["cluster_cell_hitsTruthE", "cluster_cell_hitsTruthIndex"]

# Load the branches for the specified event
first_index = [50]
items = queryTree.arrays(queries, entry_start=first_index[0], entry_stop=first_index[0] + 1)

# Access the data for each branch
hits_truth_e = items[queries[0]]
hits_truth_index = items[queries[1]]

# Print the data and its length for the first event
print("Hits Truth Energy:", hits_truth_e)
print("Length of Hits Truth Energy for the first event:", len(hits_truth_e[0][0]))

print("Hits Truth Index:", hits_truth_index)
print("Length of Hits Truth Index for the first event:", len(hits_truth_index[0][0]))


Hits Truth Energy: [[[[9.61e+03], [501], [6.28e+03], [7.04], [6.02e+03], ..., [], [], [], [], []]]]
Length of Hits Truth Energy for the first event: 24
Hits Truth Index: [[[[2], [2], [2], [2], [2], [2], [2], [2], ..., [2], [], [], [], [], [], []]]]
Length of Hits Truth Index for the first event: 24
